In [23]:
import json
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
from benchmark.utils.metadata import read_preproc, read_python, read_lean

In [24]:
def collate_metadata(base_path: str) -> pd.DataFrame:
    """
    Collates metadata.json files across indices into a single pandas DataFrame.
    
    Parameters:
    base_path (str): Base path containing indexed directories with metadata.json files
    
    Returns:
    pandas.DataFrame: Table with metadata information across all indices
    """
    # Initialize lists to store data
    records = []
    
    # Iterate through all metadata.json files
    base_dir = Path(base_path)
    for idx_dir in sorted(base_dir.glob("*")):
        if not idx_dir.is_dir():
            continue

        preproc = read_preproc(idx_dir)
        python = read_python(idx_dir)
        lean = read_lean(idx_dir)

        # Create a flat record
        record = {
            "idx": idx_dir.name, 
            "preproc_loops": preproc["loops"],
            "python_loops": python["loops"], 
            "lean_loops": lean["loops"],
            "preproc_success": preproc["latest_run_success"],
            "python_success": python["latest_run_success"],
            "lean_success": lean["latest_run_success"]
        }
        records.append(record)
    
    # Create DataFrame
    return pd.DataFrame(records).set_index("idx")

In [25]:
up = Path("..")
path = up / up / up / "artefacts" / "apps" / "train"

In [26]:
df = collate_metadata(path)

In [27]:
df

,preproc_loops,python_loops,lean_loops,preproc_success,python_success,lean_success
idx,,,,,,
0000,0,1,3,True,True,True
0001,0,1,2,True,True,True
0002,0,2,0,True,True,True
0003,2,1,1,True,True,True
0004,0,2,0,True,True,True
...,...,...,...,...,...,...
4995,0,1,0,True,True,True
4996,0,1,2,True,True,True
4997,2,0,2,True,True,True


In [28]:
df.mean()

preproc_loops      0.3390
python_loops       0.8612
lean_loops         1.3076
preproc_success    0.9878
python_success     0.9804
lean_success       0.9430
dtype: float64

In [29]:
df.sum()

preproc_loops      1695
python_loops       4306
lean_loops         6538
preproc_success    4939
python_success     4902
lean_success       4715
dtype: int64

In [30]:
df[df.lean_success == False]

,preproc_loops,python_loops,lean_loops,preproc_success,python_success,lean_success
idx,,,,,,
0062,0,0,5,True,True,False
0106,0,0,5,True,True,False
0158,0,1,5,True,True,False
0188,0,0,5,True,True,False
0258,0,5,0,True,False,False
...,...,...,...,...,...,...
4971,0,0,5,True,True,False
4973,5,0,0,False,False,False
4980,0,3,5,True,True,False


In [31]:
df.python_loops.value_counts()

python_loops
0    2113
1    1986
2     603
3     150
4      76
5      72
Name: count, dtype: int64

In [32]:
df[df.python_loops == 0]

,preproc_loops,python_loops,lean_loops,preproc_success,python_success,lean_success
idx,,,,,,
0009,0,0,2,True,True,True
0012,0,0,1,True,True,True
0013,0,0,0,True,True,True
0016,0,0,5,True,True,True
0019,0,0,1,True,True,True
...,...,...,...,...,...,...
4992,0,0,3,True,True,True
4993,0,0,0,True,True,True
4994,0,0,0,True,True,True


In [33]:
df.lean_loops.value_counts()

lean_loops
0    1833
1    1450
2     842
3     394
5     298
4     183
Name: count, dtype: int64

In [34]:
df[df.lean_success == False]

,preproc_loops,python_loops,lean_loops,preproc_success,python_success,lean_success
idx,,,,,,
0062,0,0,5,True,True,False
0106,0,0,5,True,True,False
0158,0,1,5,True,True,False
0188,0,0,5,True,True,False
0258,0,5,0,True,False,False
...,...,...,...,...,...,...
4971,0,0,5,True,True,False
4973,5,0,0,False,False,False
4980,0,3,5,True,True,False


In [50]:
df.to_csv("df.csv")

# Viz

In [49]:
plt.boxplot(df[["python_loops", "lean_loops"]])

{'whiskers': [<matplotlib.lines.Line2D at 0x7f364b9b9d30>,
 'caps': [<matplotlib.lines.Line2D at 0x7f364b9b9040>,
 'boxes': [<matplotlib.lines.Line2D at 0x7f364b9ba2a0>,
 'medians': [<matplotlib.lines.Line2D at 0x7f364b9b9430>,
 'fliers': [<matplotlib.lines.Line2D at 0x7f364b9b9700>,
 'means': []}